In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire.utils import decode

In [3]:
#Variables
website = 'https://jardinzoo.cl/perros-165'
fileName = 'dogsProducts'
folder = 'jardinzoo'

In [4]:
chrome_options = Options()
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
chrome_options.add_argument('--auto-open-devtools-for-tabs')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable_encoding')

driver = webdriver.Chrome(options=chrome_options)

driver.execute_cdp_cmd("Network.enable", {})

driver.get(website)

wait = WebDriverWait(driver, 120)

In [5]:
#Products
products = []

In [6]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time
import random

def scan_products():
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article')))
    items = driver.find_elements(By.CSS_SELECTOR, 'article')
    
    for item in items:
        image_url = item.find_element(By.CSS_SELECTOR, 'img').get_attribute('src')
        product_url = item.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        title = item.find_element(By.CSS_SELECTOR, '.product-title').text
        price = item.find_element(By.CSS_SELECTOR, '.price').text
        products.append({
            'image_url': image_url,
            'product_url': product_url,
            'title': title,
            'price': price,
        })
    
def go_to_next_page():
    try:
        next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.next')))
        next_button.click()
        wait.until(EC.staleness_of(next_button))
        return True
    except NoSuchElementException:
        print("Botón 'next' no encontrado, final de la paginación.")
        return False
    except StaleElementReferenceException:
        print("El botón 'next' ya no es válido en el DOM.")
        return False
        
flag = True
while(flag):
    try:
        scan_products()
        time.sleep(random.uniform(3, 8))
        flag = go_to_next_page()
    except Exception as e:
        print(f"Error inesperado: {e}")
        flag = False
                

Error inesperado: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF79064B095+29557]
	(No symbol) [0x00007FF7905BFA50]
	(No symbol) [0x00007FF79047B56A]
	(No symbol) [0x00007FF7904CF695]
	(No symbol) [0x00007FF7904CF8EC]
	(No symbol) [0x00007FF79051B777]
	(No symbol) [0x00007FF7904F71CF]
	(No symbol) [0x00007FF79051851C]
	(No symbol) [0x00007FF7904F6F33]
	(No symbol) [0x00007FF7904C116F]
	(No symbol) [0x00007FF7904C22D1]
	GetHandleVerifier [0x00007FF79097C96D+3378253]
	GetHandleVerifier [0x00007FF7909C8497+3688311]
	GetHandleVerifier [0x00007FF7909BD1CB+3642539]
	GetHandleVerifier [0x00007FF79070A6B6+813462]
	(No symbol) [0x00007FF7905CAB5F]
	(No symbol) [0x00007FF7905C6B74]
	(No symbol) [0x00007FF7905C6D10]
	(No symbol) [0x00007FF7905B5C1F]
	BaseThreadInitThunk [0x00007FF9D0D2257D+29]
	RtlUserThreadStart [0x00007FF9D2D2AF28+40]



In [7]:
print(f"{len(products)} Productos Encontrados!")
print(products)

1207 Productos Encontrados!
[{'image_url': 'https://jardinzoo.cl/8308-home_default/acana-dog-classic-wild-coast-2kg.jpg', 'product_url': 'https://jardinzoo.cl/adulto/3849-acana-dog-classic-wild-coast-2kg.html', 'title': 'ACANA DOG CLASSIC WILD COAST 2KG', 'price': '$ 19.200'}, {'image_url': 'https://jardinzoo.cl/8310-home_default/acana-dog-classic-wild-coast-97-kg.jpg', 'product_url': 'https://jardinzoo.cl/adulto/4432-acana-dog-classic-wild-coast-97-kg-064992562120.html', 'title': 'ACANA DOG CLASSIC WILD COAST 9.7 KG', 'price': '$ 60.200'}, {'image_url': 'https://jardinzoo.cl/8311-home_default/acana-dog-wild-atlantic-21kg.jpg', 'product_url': 'https://jardinzoo.cl/adulto/3853-acana-dog-wild-atlantic-21kg.html', 'title': 'ACANA DOG FREE RUN POULTRY 5,9KG', 'price': '$ 43.650'}, {'image_url': 'https://jardinzoo.cl/8043-home_default/acana-dog-free-run-poultry-113-kg.jpg', 'product_url': 'https://jardinzoo.cl/adulto/3640-acana-dog-free-run-poultry-113-kg.html', 'title': 'ACANA DOG FREE-RUN

In [8]:
def go_to_product(product):
    wait = WebDriverWait(driver, 120)
    driver.get(product['product_url'])
    try:
        product_reference = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.product-reference')))
        product['ean'] = product_reference.find_element(By.CSS_SELECTOR, 'span').text
    except NoSuchElementException:
        print(f"El elemento {product['title']} '.product-reference' no se encuentra en esta página")
        product['ean'] = ''

    try:
        product['brand'] = driver.find_element(By.CSS_SELECTOR, '.product-manufacturer').find_element(By.CSS_SELECTOR, 'a').text
    except NoSuchElementException:
        print(f"El elemento {product['title']} '.product-manufacturer' no se encuentra en esta página")
        product['brand'] = ''
        
    product['price'] = driver.find_element(By.CSS_SELECTOR, '.current-price').find_element(By.CSS_SELECTOR, 'span').get_attribute('content')
    product['currency'] = 'CLP'

for product in products:
    try:
        go_to_product(product)
    except Exception as e:
        print(f"Error inesperado producto {product['title']}: {e}")

El elemento ALIMENTADOR AUTOMÁTICO 5.5LT '.product-manufacturer' no se encuentra en esta página
El elemento ALL FOR PAWS TRAVEL DOG L '.product-manufacturer' no se encuentra en esta página
El elemento ALL FOR PAWS TRAVEL DOG S '.product-manufacturer' no se encuentra en esta página
El elemento ALL FOR PAWS TRAVEL DOG XL '.product-manufacturer' no se encuentra en esta página
El elemento APETIPET JARABE 100ML '.product-manufacturer' no se encuentra en esta página
El elemento ARTRI TABS 60 COMP '.product-manufacturer' no se encuentra en esta página
El elemento BABY PETS TRAINING PADS LAVANDA 50 UNIDADES '.product-manufacturer' no se encuentra en esta página
El elemento BEEF STICK BURGER 18 GRS '.product-manufacturer' no se encuentra en esta página
El elemento BEEF STICK CORDERO 12 GRS '.product-manufacturer' no se encuentra en esta página
El elemento BEEF STICK HYPOALLERGENIC 12 GRS '.product-manufacturer' no se encuentra en esta página
El elemento BEEF STICK MINERAL 12 GRS '.product-manuf

In [9]:
with open(f"{folder}/{fileName}.json", 'w', encoding='utf-8') as json_file:
    json.dump(products, json_file, ensure_ascii=False, indent=4)

print("Archivo JSON generado correctamente.")

Archivo JSON generado correctamente.


In [10]:
driver.quit()